# Análise exploratória

## Entendendo os dados


Aqui vamos dar uma olhada no conjunto de dados que temos, utilizando o polars.


Disclaimers importantes:

- Percebi que os dados de String não estão em UTF-8. Logo, no processo de E(T)L, a primeira coisa a ser feita é essa transformação.

## Dataset 1: Despesas

In [30]:
import polars as pl
import pandas as pd

pl.Config(tbl_rows=100, tbl_cols=10, fmt_str_lengths=150, fmt_float="full")

In [31]:
data_despesas = pl.scan_csv('../data_engineer_test_v2/gdvDespesasExcel.csv', encoding='utf8-lossy')

## Problema nº 1

Aqui temos um encoding utf8-lossy, que significa que os caracteres não encontrados vão ser substituídos por uma `�`.

sem o -lossy, não foi possível nem fazer a leitura.

(Esse processo ocorreu antes da definição do Disclaimer 1)

In [32]:
data_despesas.select(
    pl.col("Despesa").filter(pl.col("Despesa").str.contains("�"))
).fetch(n_rows=100)

Despesa
str
"""31900125 - COMPLEMENTACAO DE APOSENTADORIA - 13�SALARIO"""
"""31900326 - COMPLEMENTACAO DE PENSAO - 13� SALARIO"""
"""31909227 - INATIVOS DE EXERC�CIOS ANTERIORES"""
"""31910843 - AUX�LIO-RECLUS�O (LC 1.012/07)"""
"""31919224 - DESPESAS DE PESSOAL DE EXERC�CIOS ANTERIORES"""


Notamos aqui que o UTF-8 não consegue lidar com caracteres especiais como ã, é, í, º, entre outros. Sabendo disso, bora testar uma outra técnica.

In [33]:
data_despesas = pl.read_csv('../data_engineer_test_v2/gdvDespesasExcel.csv',encoding="ISO-8859-1")

In [34]:
data_despesas.select(
    pl.col("Despesa").filter(pl.col("Despesa").str.contains("�"))
).head(5)

Despesa
str


Problema nº 1 resolvido. (O meu VSCode também não sabe lidar com esses caracteres, então eu tive que olhar pelo editor de texto do próprio Github)

## Problema nº 2

Agora vamos normalizar os dados de $$.

No nosso dataset, os dados seguem o formato brasileiro, utilizando `.` para separar casa de milhar e `,` pra separar decimal.

Sabendo que isso não vai nos permitir transformar esse valor em Float, vamos deixar definido o tratamento para essa variável.

Também lembrando de tirar os espaços em branco.

In [117]:
despesas = (
    data_despesas.with_columns(
        pl.col("Liquidado")
        .map_elements(
            lambda x: round(
                float(x.strip().replace(".", "").replace(",", ".")) * 5.1945, 2
            ),
            return_dtype=pl.Float64,
        )
        .alias("liquidado"),
        pl.col("Fonte de Recursos")
        .str.splitn(" - ", 2)
        .struct.field("field_0")
        .alias("id_fonte_recurso"),
        pl.col("Fonte de Recursos")
        .str.splitn(" - ", 2)
        .struct.field("field_1")
        .alias("nome_fonte_recurso"),
    )
    .filter(~pl.col("Despesa").str.contains("TOTAL"))
    .select(
        pl.col("id_fonte_recurso"),
        pl.col("nome_fonte_recurso"),
        pl.col("Despesa").alias("despesa"),
        pl.col("liquidado")
    )
)

## Análisando

In [118]:
despesas.describe()

statistic,id_fonte_recurso,nome_fonte_recurso,despesa,liquidado
str,str,str,str,f64
"""count""","""2034""","""2034""","""2034""",2034
"""null_count""","""0""","""0""","""0""",0
"""mean""",null,null,null,658883487.4011654
"""std""",null,null,null,6463541313.142417
"""min""","""001""","""F.E.D - CREDITO POR SUPERAVIT FINANCEIRO""","""31204101 - COMPENSACAO PREVIDENCIARIA""",0.52
"""25%""",null,null,null,350976.78
"""50%""",null,null,null,4641105.29
"""75%""",null,null,null,53132716.61
"""max""","""087""","""TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR-INTRA""","""46917111 - AMORTIZ.DA DIVIDA EXTERNA CONTRATADA""",192675562112.62


Vemos aqui que não temos campos nulos, então para nós é excelente. Não nos preocuparemos em preencher dados ausentes nem nada do tipo.

Explicação:

Tentei fazer o calculo do Total Liquidado usando encadeamento de métodos, usando 

```python

pl.col("Liquidado").str.strip_char_start().str.replace(".", "").replace(",", ".")
```

Mas essa abordagem não estava dando resultado. Por algum motivo, o Polars parava no primeiro método (`strip_char_start()`) Ou simplsmente comia um char da string. usando `map_elements()` não é tão performático, mas para nível de análise, tá valendo.

In [119]:
despesas.tail(5)

id_fonte_recurso,nome_fonte_recurso,despesa,liquidado
str,str,str,f64
"""087""","""OP.CRED.CONTR.EXTERIOR-DOT.INIC.CR.SUP-INTRA""","""44905111 - GERENCIAMENTO DE OBRAS""",42335474.51
"""087""","""OP.CRED.CONTR.EXTERIOR-DOT.INIC.CR.SUP-INTRA""","""44905112 - INSTALACOES""",13220507.61
"""087""","""OP.CRED.CONTR.EXTERIOR-DOT.INIC.CR.SUP-INTRA""","""44905130 - EXECUCAO DE OBRAS E INSTALACOES""",170535224.05
"""087""","""OP.CRED.CONTR.EXTERIOR-DOT.INIC.CR.SUP-INTRA""","""44905210 - VEICULOS DIVERSOS""",3922934.5
"""087""","""OP.CRED.CONTR.EXTERIOR-DOT.INIC.CR.SUP-INTRA""","""44905234 - OUTROS EQUIPAMENTOS E MATERIAL PERMANENTE""",10900658.25


In [120]:
despesas.tail(5)

id_fonte_recurso,nome_fonte_recurso,despesa,liquidado
str,str,str,f64
"""087""","""OP.CRED.CONTR.EXTERIOR-DOT.INIC.CR.SUP-INTRA""","""44905111 - GERENCIAMENTO DE OBRAS""",42335474.51
"""087""","""OP.CRED.CONTR.EXTERIOR-DOT.INIC.CR.SUP-INTRA""","""44905112 - INSTALACOES""",13220507.61
"""087""","""OP.CRED.CONTR.EXTERIOR-DOT.INIC.CR.SUP-INTRA""","""44905130 - EXECUCAO DE OBRAS E INSTALACOES""",170535224.05
"""087""","""OP.CRED.CONTR.EXTERIOR-DOT.INIC.CR.SUP-INTRA""","""44905210 - VEICULOS DIVERSOS""",3922934.5
"""087""","""OP.CRED.CONTR.EXTERIOR-DOT.INIC.CR.SUP-INTRA""","""44905234 - OUTROS EQUIPAMENTOS E MATERIAL PERMANENTE""",10900658.25


Resolvido!

## Dataset 2: Receitas

In [121]:
data_receitas = pl.read_csv('../data_engineer_test_v2/gdvReceitasExcel.csv',
                            encoding="ISO-8859-1")

In [122]:
data_receitas.head(10)

Fonte de Recursos,Receita,Arrecadado
str,str,str
"""001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR""","""11130111 - IRRF - IMP.RENDA PESSOA FISICA-PRINCIPAL""","""4267858335,83"""
"""001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR""","""11130311 - IRRF-IMP. RENDA RET. FTE-TRABALHO-PRINCIPAL""","""4323772997,93"""
"""001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR""","""11130341 - IRRF-OUTROS RENDIMENTOS-PRINCIPAL""","""123071158,14"""
"""001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR""","""11180121 - IPVA - IMP. PROPR. VEIC. AUTOM - PRINCIPAL""","""8202995933,47"""
"""001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR""","""11180122 - IPVA - MULTAS E JUROS""","""6927,39"""
"""001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR""","""11180123 - IPVA - DIVIDA ATIVA""","""235614546,72"""
"""001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR""","""11180124 - IPVA - MULTA E JUROS-DIVIDA ATIVA""","""401195,68"""
"""001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR""","""11180131 - ITCMD-IMP.TRA.CAUSA MORTIS/DOACAO-PRINCIPAL""","""3158253957,69"""
"""001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR""","""11180132 - ITCMD - MULTAS E JUROS""","""218862721,73"""


Os tratamentos serão bem parecidos com o que tivemos no caso 1. Temos uma String com o ID e o Nome no campo `Fonte de Recursos` e um valor parcialmente formatado no campo `Arrecadado`.

In [123]:
receitas = (
    data_receitas.with_columns(
        pl.col("Arrecadado")
        .map_elements(
            lambda x: round(
                float(x.strip().replace(".", "").replace(",", ".")) * 5.1945, 2
            ),
            return_dtype=pl.Float64,
        )
        .alias("arrecadado"),
        pl.col("Fonte de Recursos")
        .str.splitn(" - ", 2)
        .struct.field("field_0")
        .alias("id_fonte_recurso"),
        pl.col("Fonte de Recursos")
        .str.splitn(" - ", 2)
        .struct.field("field_1")
        .alias("nome_fonte_recurso"),
    )
    .filter(~pl.col("Receita").str.contains("TOTAL"))
    .select(
        pl.col("id_fonte_recurso"),
        pl.col("nome_fonte_recurso"),
        pl.col("Receita").alias("receita"),
        pl.col("arrecadado")
    )
)

In [124]:
receitas.describe()

statistic,id_fonte_recurso,nome_fonte_recurso,receita,arrecadado
str,str,str,str,f64
"""count""","""284""","""284""","""284""",284
"""null_count""","""0""","""0""","""0""",0
"""mean""",null,null,null,4708772247.468557
"""std""",null,null,null,35655264030.06082
"""min""","""001""","""EXTRA ORCAMENTARIA""","""11130111 - IRRF - IMP.RENDA PESSOA FISICA-PRINCIPAL""",0
"""25%""",null,null,null,506731.53
"""50%""",null,null,null,40876775.8
"""75%""",null,null,null,957549126.76
"""max""","""099""","""TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR-INTRA""","""89900011 - DEMAIS RECEITAS DE CAPITAL-PRINCIPAL INTRA""",557118164162.74


Agora temos que usar a API `awesomeapi.com.br/api-de-moedas` para transformar em real os dados da coluna Total Arrecadado (estão em USD com a cotação máxima do dolár do dia 22/06/2022).

## Join

Vamos juntar as duas tabelas para ver o que temos?

In [125]:
merged = (despesas
          .select(["id_fonte_recurso","nome_fonte_recurso", "liquidado"])
          .group_by(["id_fonte_recurso", "nome_fonte_recurso"])
          .agg(pl.col("liquidado").sum())
          .with_columns(pl.col("liquidado").round(2))
          .join(
            receitas
                .select(["nome_fonte_recurso", "arrecadado"])
                .group_by("nome_fonte_recurso")
                .agg(pl.col("arrecadado").sum())
                .with_columns(pl.col("arrecadado").round(2)),
            "nome_fonte_recurso"
        ))

In [126]:
merged.sort(["liquidado", "arrecadado"], descending=True)

id_fonte_recurso,nome_fonte_recurso,liquidado,arrecadado
str,str,f64,f64
"""001""","""TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR""",761629054293.4,755069075741.17
"""002""","""RECURSOS VINCULADOS ESTADUAIS""",265605878631.16,269273764451.94
"""081""","""TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR-INTRA""",160672067571.69,160756796425.25
"""004""","""REC.PROPRIO-ADM.IND.-DOT.INIC.CR.SUPL.""",52974388171.73,56511589734.27
"""005""","""RECURSOS VINCULADOS FEDERAIS""",42029087040.35,44597592235.13
"""003""","""RECURSOS VINCULADOS-FUNDO ESPECIAL DE DESPES""",22669631623.25,28276722671.9
"""007""","""OP.CRED.E CONTRIB.DO EXTERIOR-DOT.INIC.CR.SU""",7862831626.57,12376224513.05
"""041""","""TESOURO - CREDITO POR SUPERAVIT FINANCEIRO""",7362312397.5,0.05
"""047""","""REC.OPERAC. DE CREDITO-P/SUPERAVIT FINANCEIR""",4638492444.03,247368672.37


In [65]:
merged.filter(pl.col("id_fonte_recurso") == "001").select(
    pl.col("id_fonte_recurso"),
    pl.col("nome_fonte_recurso"),
    pl.col("receita"),
    pl.col("arrecadado"),
    pl.col("despesa"),
    pl.col("liquidado"),
)

id_fonte_recurso,nome_fonte_recurso,receita,arrecadado,despesa,liquidado
str,str,str,f64,str,f64
"""001""","""TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR""","""11130111 - IRRF - IMP.RENDA PESSOA FISICA-PRINCIPAL""",22619649179.9,"""31900119 - ATRASADOS-OUTROS PODERES/MINIST.PUBLICO""",422730674.75
"""001""","""TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR""","""11130311 - IRRF-IMP. RENDA RET. FTE-TRABALHO-PRINCIPAL""",22915996889.03,"""31900119 - ATRASADOS-OUTROS PODERES/MINIST.PUBLICO""",422730674.75
"""001""","""TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR""","""11130341 - IRRF-OUTROS RENDIMENTOS-PRINCIPAL""",652277138.14,"""31900119 - ATRASADOS-OUTROS PODERES/MINIST.PUBLICO""",422730674.75
"""001""","""TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR""","""11180121 - IPVA - IMP. PROPR. VEIC. AUTOM - PRINCIPAL""",43475878447.39,"""31900119 - ATRASADOS-OUTROS PODERES/MINIST.PUBLICO""",422730674.75
"""001""","""TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR""","""11180122 - IPVA - MULTAS E JUROS""",36715.17,"""31900119 - ATRASADOS-OUTROS PODERES/MINIST.PUBLICO""",422730674.75
"""001""","""TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR""","""11180123 - IPVA - DIVIDA ATIVA""",1248757097.62,"""31900119 - ATRASADOS-OUTROS PODERES/MINIST.PUBLICO""",422730674.75
"""001""","""TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR""","""11180124 - IPVA - MULTA E JUROS-DIVIDA ATIVA""",2126337.1,"""31900119 - ATRASADOS-OUTROS PODERES/MINIST.PUBLICO""",422730674.75
"""001""","""TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR""","""11180131 - ITCMD-IMP.TRA.CAUSA MORTIS/DOACAO-PRINCIPAL""",16738745975.76,"""31900119 - ATRASADOS-OUTROS PODERES/MINIST.PUBLICO""",422730674.75
"""001""","""TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR""","""11180132 - ITCMD - MULTAS E JUROS""",1159972425.17,"""31900119 - ATRASADOS-OUTROS PODERES/MINIST.PUBLICO""",422730674.75


Developed by

```shell
             ___         __  __              
            /   |  _____/ /_/ /_  __  _______
           / /| | / ___/ __/ __ \/ / / / ___/
          / ___ |/ /  / /_/ / / / /_/ / /    
         /_/  |_/_/   \__/_/ /_/\__,_/_/     
```